In [22]:
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
import pickle
import pandas as pd
import math #this is used in graphic section to use the irrational number pi
import matplotlib.pyplot as plt
%matplotlib inline 
output_notebook()

Loading BokehJS ...

In [23]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

page = requests.get(gdelt_base_url+'index.html') #Grab GDELT reference list which is by day
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href") #Returns all the possible CSV files of GDELT data as a references list

# separate out those links that begin with four digits 
'''
Will extract just the days resulting in list like: 
['20200617.export.CSV.zip',
 '20200616.export.CSV.zip',
 '20200615.export.CSV.zip',...]
 Until 2015
'''

file_list = [x for x in link_list if str.isdigit(x[0:4])]
file_list

['20201010.export.CSV.zip',
 '20201009.export.CSV.zip',
 '20201008.export.CSV.zip',
 '20201007.export.CSV.zip',
 '20201006.export.CSV.zip',
 '20201005.export.CSV.zip',
 '20201004.export.CSV.zip',
 '20201003.export.CSV.zip',
 '20201002.export.CSV.zip',
 '20201001.export.CSV.zip',
 '20200930.export.CSV.zip',
 '20200929.export.CSV.zip',
 '20200928.export.CSV.zip',
 '20200927.export.CSV.zip',
 '20200926.export.CSV.zip',
 '20200925.export.CSV.zip',
 '20200924.export.CSV.zip',
 '20200923.export.CSV.zip',
 '20200922.export.CSV.zip',
 '20200921.export.CSV.zip',
 '20200920.export.CSV.zip',
 '20200919.export.CSV.zip',
 '20200918.export.CSV.zip',
 '20200917.export.CSV.zip',
 '20200916.export.CSV.zip',
 '20200915.export.CSV.zip',
 '20200914.export.CSV.zip',
 '20200913.export.CSV.zip',
 '20200912.export.CSV.zip',
 '20200911.export.CSV.zip',
 '20200910.export.CSV.zip',
 '20200909.export.CSV.zip',
 '20200908.export.CSV.zip',
 '20200907.export.CSV.zip',
 '20200906.export.CSV.zip',
 '20200905.export.CS

In [24]:
#Counters to help assess how many files are coming and going out
infilecounter = 0
outfilecounter = 0

In [25]:
import os.path #To help navigate the file directories
import urllib #To request from GDELT
import zipfile #TO unzip the files we downlaod
import glob #To go through multiple files in a directory
import operator 

local_path = './results/' # Will save to empy results folder to help keep file clean

fips_country_code = 'TW'  ## Exploring Taiwan

#Adjust list number to get days wanted 
for compressed_file in file_list[0:10]: #!!!!!Only getting index 0 to 6!!!!!!
    print(compressed_file,)
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('downloading,'),
        urllib.request.urlretrieve(url=gdelt_base_url+compressed_file, 
                           filename=local_path+compressed_file)
        
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,'),
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+fips_country_code+'%04i.tsv'%outfilecounter
        
        # open the infile and outfile
        with open(infile_name, mode='r', encoding="ISO-8859-1") as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                    outfile.write(line)
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done', infilecounter)
    

20201010.export.CSV.zip
extracting,
parsing,
done 1
20201009.export.CSV.zip
extracting,
parsing,
done 2
20201008.export.CSV.zip
extracting,
parsing,
done 3
20201007.export.CSV.zip
extracting,
parsing,
done 4
20201006.export.CSV.zip
extracting,
parsing,
done 5
20201005.export.CSV.zip
extracting,
parsing,
done 6
20201004.export.CSV.zip
extracting,
parsing,
done 7
20201003.export.CSV.zip
extracting,
parsing,
done 8
20201002.export.CSV.zip
extracting,
parsing,
done 9
20201001.export.CSV.zip
extracting,
parsing,
done 10


In [26]:
import pandas as pd

# Get the GDELT field names from a helper file
colnames = pd.read_csv('CSV.header.fieldids.csv')['Field Name']


# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+fips_country_code+'*')
DFlist = []
for active_file in files:
    print(active_file)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID'], encoding='iso-8859-1'))

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path+'backup'+fips_country_code+'.pickle')    
    
# once everythin is safely stored away, remove the temporary files
for active_file in files:
    os.remove(active_file)

./results/IN0000.tsv
./results/IN0001.tsv
./results/IN0002.tsv
./results/IN0003.tsv
./results/IN0004.tsv
./results/IN0005.tsv
./results/IN0006.tsv
./results/IN0007.tsv
./results/IN0008.tsv
./results/IN0009.tsv


In [27]:
India_Data = pd.read_pickle(r"./results/backupIN.pickle")

In [28]:
India_Data.head()

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
951015558,20191011,201910,2019,2019.7699,LAB,EMPLOYEE,NaN,NaN,NaN,NaN,...,NaN,4,"Bengaluru, Karnataka, India",IN,IN19,12.9833,77.5833,-2090174,20201010,https://katcountry989.com/2020/10/09/microsoft...
951015559,20191011,201910,2019,2019.7699,LAB,EMPLOYEE,NaN,NaN,NaN,NaN,...,-2090174,4,"Bengaluru, Karnataka, India",IN,IN19,12.9833,77.5833,-2090174,20201010,https://katcountry989.com/2020/10/09/microsoft...
951015617,20201010,202010,2020,2020.7671,NaN,NaN,NaN,NaN,NaN,NaN,...,6210381,4,"Haryana, Haryana, India",IN,IN10,29.6161,76.9806,6210381,20201010,https://indianexpress.com/article/cities/chand...
951015620,20201010,202010,2020,2020.7671,NaN,NaN,NaN,NaN,NaN,NaN,...,-2106102,4,"New Delhi, Delhi, India",IN,IN07,28.6,77.2,-2106102,20201010,https://www.malaysiasun.com/news/266644434/ind...
951015621,20201010,202010,2020,2020.7671,NaN,NaN,NaN,NaN,NaN,NaN,...,BG,4,"New Delhi, Delhi, India",IN,IN07,28.6,77.2,-2106102,20201010,https://www.malaysiasun.com/news/266644434/ind...


In [29]:
India_Data.columns

Index(['SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code',
       'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long',
       'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat',
       'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type',
      

In [30]:
def ref_dict(df):
    cols = list(df)
    ref_dict = {}
    for row in df.iterrows(): 
        ref_dict[row[1][cols[0]]] = row[1][cols[1]]
    
    return ref_dict

In [31]:
#Read in event codes
eventCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.eventcodes.txt", sep='\t'))
#Read in Goldsteinscale
goldScale = ref_dict(pd.read_csv("./Ref Codes/CAMEO.goldsteinscale.txt", sep='\t'))
#Read in ethnic groups
ethnicCodes =ref_dict(pd.read_csv("./Ref Codes/CAMEO.ethnic.txt", sep='\t'))
#Read in known Groups
knownGroups = ref_dict(pd.read_csv("./Ref Codes/CAMEO.knowngroup.txt", sep='\t'))
#Read in relgion
religionCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.religion.txt", sep='\t'))
#Read in type
typeCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.type.txt", sep='\t'))

eventCodes

{1: 'MAKE PUBLIC STATEMENT',
 10: 'DEMAND',
 11: 'DISAPPROVE',
 12: 'REJECT',
 13: 'THREATEN',
 14: 'PROTEST',
 15: 'EXHIBIT FORCE POSTURE',
 16: 'REDUCE RELATIONS',
 17: 'COERCE',
 18: 'ASSAULT',
 19: 'FIGHT',
 2: 'APPEAL',
 20: 'USE UNCONVENTIONAL MASS VIOLENCE',
 21: 'Appeal for material cooperation, not specified below',
 211: 'Appeal for economic cooperation',
 212: 'Appeal for military cooperation',
 213: 'Appeal for judicial cooperation',
 214: 'Appeal for intelligence',
 22: 'Appeal for diplomatic cooperation, such as policy support',
 23: 'Appeal for aid, not specified below',
 231: 'Appeal for economic aid',
 232: 'Appeal for military aid',
 233: 'Appeal for humanitarian aid',
 234: 'Appeal for military protection or peacekeeping',
 24: 'Appeal for political reform, not specified below',
 241: 'Appeal for change in leadership',
 242: 'Appeal for policy change',
 243: 'Appeal for rights',
 244: 'Appeal for change in institutions, regime',
 25: 'Appeal to yield',
 251: 'Appeal 

In [32]:
# Turn colnames into list for ref

cross_ref = list(colnames)

In [33]:
# Create look up table to get values instead of numbers

look_up_code = {"eventCodes": [26,27,28], "goldScale":[30], "ethnicCodes":[9,19], "knownGroups":[8,18], 
                "religionCodes":[10,11,20,21], "typeCodes":[12,13,14,22,23,24]}

In [34]:
'''
Helper function to user can reorient data based on interest from codes

data: India_Data - pandas dataframe
ref: key value from look_look_code - string
codebook: reference 
'''

import math

def search_dict(data,ref, codebook):
    res = {}
    look_up = look_up_code[ref]
    col_names = []
    for i in look_up: 
        col_names.append(cross_ref[i])
    
    for col in col_names: 
        for row in data.iterrows(): 
            if isinstance(row[1][col],float):
                #print (type(row[1][col]), col)
                pass
            else: 
                #print (col)
                var = codebook[row[1][col]].upper()
                #print (var, row[1][col])
                if var in res.keys(): 
                    #print(row[1][col])
                    res[var].append(dict(row[1]))
                else: 
                    res[var] = [dict(row[1])]
    return res

In [35]:
res = search_dict(India_Data, "ethnicCodes", ethnicCodes)
res.keys()

dict_keys(['DALIT', 'GUJARATI', 'PAHARI RAJPUT', 'SAMI', 'TAMIL', 'INDIGENOUS', 'SENA', 'TIBETAN', 'AFAR', 'BIHARI', 'BENGALI-HINDU', 'BHOJPURI', 'KASHMIRI', 'KANNADA', 'ADIVASI', 'BETI-PAHUIN', 'MARATHI', 'MANDE', 'TELUGU', 'NATIVE AMERICAN', 'TIGRE', 'HUTU', 'MALAYALAM', 'MANCHU', 'PUNJABI', 'NAGA', 'RAKHINE', 'CHINOOK', 'MALDIVIAN', 'BLACK-AFRICAN', 'APACHE', 'MARI', 'DRAVIDIAN', 'URDU', 'MIAO', 'BALOCH', 'NEPALI', 'UYGHUR', 'MENDE', 'GONDI', 'SWAHILI', 'HIMACHALI', 'EAST INDIAN', 'MANDAR', 'MADHESI', 'ASSAMESE', 'PERSIAN', 'SINHALESE', 'MUNDA', 'SHAN', 'TUTSI', 'INUIT'])

In [36]:
#verfication to ensure code is working properly
for k,v in res.items(): 
    print (k, ": ", len(v))

DALIT :  1318
GUJARATI :  26
PAHARI RAJPUT :  45
SAMI :  3
TAMIL :  51
INDIGENOUS :  41
SENA :  355
TIBETAN :  82
AFAR :  4
BIHARI :  33
BENGALI-HINDU :  44
BHOJPURI :  4
KASHMIRI :  55
KANNADA :  53
ADIVASI :  27
BETI-PAHUIN :  6
MARATHI :  73
MANDE :  2
TELUGU :  34
NATIVE AMERICAN :  2
TIGRE :  5
HUTU :  13
MALAYALAM :  30
MANCHU :  1
PUNJABI :  15
NAGA :  65
RAKHINE :  3
CHINOOK :  10
MALDIVIAN :  8
BLACK-AFRICAN :  2
APACHE :  7
MARI :  1
DRAVIDIAN :  14
URDU :  4
MIAO :  1
BALOCH :  3
NEPALI :  27
UYGHUR :  2
MENDE :  1
GONDI :  2
SWAHILI :  4
HIMACHALI :  1
EAST INDIAN :  1
MANDAR :  2
MADHESI :  2
ASSAMESE :  7
PERSIAN :  1
SINHALESE :  1
MUNDA :  1
SHAN :  3
TUTSI :  3
INUIT :  1
